In [67]:
import mysql.connector
import pymysql
import warnings
import requests
import json
import codecs
from bs4 import BeautifulSoup

In [94]:
#ignore warnings
warnings.filterwarnings("ignore")

# Extract the JSON corresponding to the first 100 contributors from the API

In [28]:
# get the json
url = "https://api.github.com/repos/apache/hadoop/contributors?&per_page=100"
params = {'per_page': 100}
page = requests.get(url, params=params)
doc = BeautifulSoup(page.content, 'html.parser')
json_dict = json.loads(str(doc))

In [40]:
# create a list to store the info
user_dict_list = []

# get user info
headers = {'User-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
user_acct = 'ur_account'
token = 'ur_tokenT'

for i in range(100):
    user = json_dict[i]['url']
    user_info = requests.get(user, headers=headers, auth = (user_acct,token))
    user_info_content = BeautifulSoup(user_info.content, 'html.parser')   
    user_dict = json.loads(str(user_info_content))
    user_dict_list.append(user_dict)

user_dict_list

[{'login': 'aajisaka',
  'id': 3403122,
  'node_id': 'MDQ6VXNlcjM0MDMxMjI=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/3403122?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/aajisaka',
  'html_url': 'https://github.com/aajisaka',
  'followers_url': 'https://api.github.com/users/aajisaka/followers',
  'following_url': 'https://api.github.com/users/aajisaka/following{/other_user}',
  'gists_url': 'https://api.github.com/users/aajisaka/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/aajisaka/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/aajisaka/subscriptions',
  'organizations_url': 'https://api.github.com/users/aajisaka/orgs',
  'repos_url': 'https://api.github.com/users/aajisaka/repos',
  'events_url': 'https://api.github.com/users/aajisaka/events{/privacy}',
  'received_events_url': 'https://api.github.com/users/aajisaka/received_events',
  'type': 'User',
  'site_admin': False,
  'name': 'Akira Ajisaka

# Extract 12 keys from the 100 json elements

In [47]:
# create another list for the 12 keys required
user_q2 = []

# append the required info
for i in range(100):
    user_q2.append((user_dict_list[i]["login"], 
                    user_dict_list[i]["id"], 
                    user_dict_list[i]["location"], 
                    user_dict_list[i]["email"], 
                    user_dict_list[i]["hireable"], 
                    user_dict_list[i]["bio"], 
                    user_dict_list[i]["twitter_username"], 
                    user_dict_list[i]["public_repos"], 
                    user_dict_list[i]["public_gists"], 
                    user_dict_list[i]["followers"], 
                    user_dict_list[i]["following"], 
                    user_dict_list[i]["created_at"]))
    
# print result to screen
user_q2

[('aajisaka',
  3403122,
  'Tokyo, Japan',
  None,
  None,
  'Software engineer @aws Glue. Apache (@apache) Hadoop/Yetus/Incubator committer and PMC member. Apache Member. I like OSS and distributed systems.',
  'ajis_ka',
  31,
  43,
  210,
  47,
  '2013-01-28T07:02:40Z'),
 ('vinoduec',
  384796,
  None,
  None,
  None,
  None,
  None,
  1,
  1,
  37,
  0,
  '2010-09-02T07:32:46Z'),
 ('szetszwo',
  907380,
  None,
  None,
  None,
  None,
  None,
  6,
  0,
  65,
  0,
  '2011-07-11T09:30:08Z'),
 ('arp7',
  2480225,
  None,
  None,
  None,
  None,
  None,
  14,
  1,
  27,
  2,
  '2012-10-03T15:51:33Z'),
 ('cnauroth',
  227407,
  'Seattle, WA',
  'cnauroth@apache.org',
  None,
  'Software engineer at Google Cloud Dataproc.  Apache Hadoop PMC member.  ASF member.  The opinions stated here are my own, not necessarily those of my employer.',
  'cnauroth',
  31,
  5,
  78,
  17,
  '2010-03-21T18:06:44Z'),
 ('jlowe',
  1360766,
  'Champaign, IL',
  None,
  None,
  None,
  None,
  9,
  0,
  19,

In [112]:
# check if all are extracted
len(user_q2)

100

# Store the data in a SQL table

In [155]:
# create DB
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",         
    password="helowrld!",
    auth_plugin='mysql_native_password'
)

In [169]:
query = "CREATE DATABASE IF NOT EXISTS SQLgit_DB CHARACTER SET utf8 DEFAULT COLLATE utf8_general_ci"
print(query)
cursor = conn.cursor()
cursor.execute(query)

CREATE DATABASE IF NOT EXISTS SQLgit_DB CHARACTER SET utf8 DEFAULT COLLATE utf8_general_ci


In [176]:
# about 'data type' choices

# "login VARCHAR(1000)"            # login uses VARCHAR(1000) likely to be a string value
# "id INT"                         # id uses INT likely to be a number  
# "location VARCHAR(1000)"         # location likely to be a string value        
# "email VARCHAR(1000)"            # email likely to be a string value        
# "hireable VARCHAR(1000)"         # hireable likely to be a string value  
# "bio VARCHAR(1000)"              # bio likely to be a string value  
# "twitter_username VARCHAR(1000)" # tweet user name likely to be a string value  
# "public_repos INT"               # public repos likely to be a number  
# "public_gists INT"               # public gists likely to be a number  
# "followers INT"                  # followers likely to be a number  
# "following INT"                  # following repos likely to be a number  
# "created_at VARCHAR(1000)"       # created_at likely to be a string value  

In [ ]:
# about 'primary key' choices
# Choosing 'id' as primary key because it's a series of number unique to each user"

In [170]:
# create a table in the database
# should specify 'CHARACTER SET utf8 DEFAULT COLLATE utf8_general_ci' to avoid limit on index length

conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",         
    password="heywrld!",
    auth_plugin='mysql_native_password',
    database = 'SQLgit_DB'
)

cursor = conn.cursor()
cursor.execute("CREATE TABLE git_100_1 (login VARCHAR(1000), id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, location VARCHAR(1000), email VARCHAR(1000),hireable VARCHAR(1000),bio VARCHAR(1000),twitter_username VARCHAR(1000),public_repos INT, public_gists INT,followers INT,following INT,created_at VARCHAR(1000)) ")

In [171]:
# insert values
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",         
    password="heywrld!",
    auth_plugin='mysql_native_password',
    database = 'SQLgit_DB'
)

for i in range(100):
    query = "INSERT INTO git_100_1 (login, id, location, email, hireable, bio, twitter_username, public_repos, public_gists, followers, following, created_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values_1 = user_q2[i]
    
    cursor = conn.cursor()
    cursor.execute(query, values_1)
    conn.commit()
    print(cursor.rowcount, "record inserted")

1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record inserted
1 record i

In [172]:
# check the inserted data

conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",         
    password="heywrld!",
    auth_plugin='mysql_native_password',
    database = 'SQLgit_DB'
)

query = "SELECT * FROM git_100_1"

# get records from the table
cursor = conn.cursor()
cursor.execute(query)

# fetch all records from the 'cursor' object
records = cursor.fetchall()

# Show the data
for record in records:
    print(record)

('toddlipcon', 14135, 'San Francisco, CA', 'todd@lipcon.org', None, None, 'tlipcon', 103, 276, 427, 8, '2008-06-18T03:04:16Z')
('xslogic', 31204, 'Sunnyvale, CA', 'asuresh@apache.org', None, None, None, 22, 6, 39, 5, '2008-10-27T08:54:58Z')
('QwertyManiac', 31518, None, None, None, None, None, 4, 0, 78, 1, '2008-10-28T20:39:50Z')
('umbrant', 33948, None, 'andrew@umbrant.com', None, None, None, 33, 4, 95, 4, '2008-11-11T23:57:45Z')
('tomwhite', 85085, 'Wales, UK', 'tom.e.white@gmail.com', None, None, None, 200, 2, 908, 2, '2009-05-15T20:30:34Z')
('jghoman', 95350, 'Seattle', None, None, None, None, 163, 10, 128, 40, '2009-06-14T21:30:03Z')
('oza', 104111, 'Tokyo, Japan', 'ozawa.tsuyoshi@gmail.com', None, None, None, 64, 84, 88, 49, '2009-07-12T11:50:13Z')
('elicollins', 131502, 'California', None, None, None, None, 6, 0, 32, 1, '2009-09-26T05:57:26Z')
('cdouglas', 145508, None, None, None, None, None, 2, 2, 21, 5, '2009-10-27T21:46:30Z')
('atm', 148338, None, None, None, None, 'atm', 5,

# Speed up look-ups of "login", "location", and "hireable".

method choice  
I choose to index all of these three keys. This is because indexing these three columns allows for binary search, and binary search speeds up the searching process. 

In [175]:
# indexing the three keys
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",         
    password="heywrld!",
    auth_plugin='mysql_native_password',
    database = 'SQLgit_DB'
)

query1 = 'ALTER TABLE `git_100_1` ADD INDEX `login_index` (`login`) using HASH;'
query2 = 'ALTER TABLE `git_100_1` ADD INDEX `location_index` (`location`) using HASH;'
query3 = 'ALTER TABLE `git_100_1` ADD INDEX `hireable_index` (`hireable`) using HASH;'

cursor = conn.cursor()
cursor.execute(query1,query2,query3)

<generator object MySQLCursor._execute_iter at 0x10e379c40>